In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('linreg').getOrCreate()

In [6]:
# cmd: pip3 install numpy
from pyspark.ml.regression import LinearRegression

In [7]:
# Data is already formatted for Machine Learning!
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [9]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [10]:
# Load training data
lr = LinearRegression(featuresCol='features',
                      labelCol='label',
                      predictionCol='prediction')

In [11]:
# Fit the model
lrModel = lr.fit(training)

In [12]:
# Print the model's coefficients
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [13]:
# Print the model's intercept
lrModel.intercept

0.14228558260358093

In [14]:
# Training summary
training_summary = lrModel.summary

In [15]:
training_summary.r2 # R2

0.027839179518600154

In [16]:
training_summary.rootMeanSquaredError # RMSE

10.16309157133015

In [18]:
# TRAIN/TEST SPLIT
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [19]:
split_object = all_data.randomSplit([0.7,0.3])

In [20]:
split_object # 2 dataframes, training and test data!

[DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector]]

In [21]:
# Realistic Method!
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [24]:
# Double-Check the split
train_data.describe().show()
test_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 352|
|   mean|-0.07990466805170236|
| stddev|   10.05993802953512|
|    min| -28.571478869743427|
|    max|   27.78383192005107|
+-------+--------------------+

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                149|
|   mean| 1.0525351884153582|
| stddev|  10.89665434909406|
|    min|-26.736207182601724|
|    max| 27.111027963108548|
+-------+-------------------+



In [25]:
# Fit the model to the training data only!
correct_model = lr.fit(train_data)

In [26]:
# Test the model on new data
test_results = correct_model.evaluate(test_data)

In [28]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-23.898786146424097|
| -21.82461723712168|
|-19.484074947719748|
|-18.327740730552563|
|-20.170483878116244|
| -19.03477898210663|
| -14.12699935939318|
| -16.37756730576434|
|-17.765953968025258|
| -16.20472972697216|
| -18.84945023767399|
|-15.013075078851973|
| -17.06160510369329|
|-16.479295353337243|
|-12.999775349739828|
|-13.748070066797121|
|-13.306473368159393|
| -7.867624441335933|
|-13.200292285853497|
| -8.796853006076946|
+-------------------+
only showing top 20 rows



10.881231286688301

In [29]:
test_results.rootMeanSquaredError # The lower, the better!

10.881231286688301

In [30]:
# Let's test the model on unlabeled data
unlabeled_data = test_data.select('features')

In [31]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [32]:
predictions = correct_model.transform(unlabeled_data)

In [33]:
predictions.show()
# Since it is unlabeled, you cannot evaluate on RMSE or r2!

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -2.8374210361776258|
|(10,[0,1,2,3,4,5,...| -1.6628228838148338|
|(10,[0,1,2,3,4,5,...| -0.5734076680694631|
|(10,[0,1,2,3,4,5,...| -1.0745952996619914|
|(10,[0,1,2,3,4,5,...|  1.3245614052176624|
|(10,[0,1,2,3,4,5,...|  0.7595654161019998|
|(10,[0,1,2,3,4,5,...| -2.8994929048163676|
|(10,[0,1,2,3,4,5,...| -0.3415295278407495|
|(10,[0,1,2,3,4,5,...|  1.5045236925725265|
|(10,[0,1,2,3,4,5,...| 0.11907068595067224|
|(10,[0,1,2,3,4,5,...|  3.0687652050506893|
|(10,[0,1,2,3,4,5,...| -0.4243097145792441|
|(10,[0,1,2,3,4,5,...|  1.7020602238606115|
|(10,[0,1,2,3,4,5,...|  1.1445278734148998|
|(10,[0,1,2,3,4,5,...| -2.3112052396764615|
|(10,[0,1,2,3,4,5,...|-0.22806086435558107|
|(10,[0,1,2,3,4,5,...| -0.5606145269993758|
|(10,[0,1,2,3,4,5,...|  -4.905602557915263|
|(10,[0,1,2,3,4,5,...|  0.6995185004984412|
|(10,[0,1,2,3,4,5,...| -3.614093